## Functions and libraries

In [6]:
# en python:
import os
os.environ["https_proxy"] = "http://proxy.fr.cfm.fr:6060"
#from nltk.tokenize import sent_tokenize
from datasets import Dataset
import numpy as np
import pandas as pd
import regex as re
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch.nn.functional as F
import seaborn as sns
import matplotlib.pyplot as plt

OSError: libcurand.so.10: cannot open shared object file: No such file or directory

In [ ]:
def translate(tokenizer, model,news='he is cute.'):
    """
    translate
    
    Args:
        batch ():
        tokenizer (Transformers.tokenizer):
        model (Transformers.model):
        
    Returns:
        The new dataset with a 'translation' column that have the translated text.
    """
    tokenized_batch = tokenizer(
        news,
        padding="longest",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    inputs = { k:v.to(device) for k, v in tokenized_batch.items() if k in tokenizer.model_input_names }
    #mem_a = torch.cuda.memory_summary(device, True)
    

    with torch.no_grad():
        translation = model.generate(**inputs)
        
        output=tokenizer.batch_decode(translation, skip_special_tokens=True)
        translation.to('cpu')
        print(translation.device)
        
    return {
        'translation': output
    }

In [ ]:
#remove space
def remove_space(text):
    text=text.replace(' ','')
    return text
#Full-width to half-width
def full_to_half(sentence):      
    change_sentence=""
    for word in sentence:
        inside_code=ord(word)
        if inside_code==12288:    #Direct conversion of full-width spaces
            inside_code=32
        elif inside_code>=65281 and inside_code<=65374:  #Full-width characters (except spaces) are converted according to the relationship
            inside_code-=65248
        change_sentence+=chr(inside_code)
    return change_sentence
# remove unnecessary symbols as well as things in paratheses     
def remove_special_chars(text):
    special_chars = r'[&*+\-\/<=>?@\^_|~]'
    # replace special characters with an empty string
    cleaned_text = re.sub(special_chars, '', text)
    pattern = r'\(.*?\)' # remove things in paratheses
    cleaned_text = re.sub(pattern, '', cleaned_text)
    return cleaned_text    
def add_chinese_period (string):
    """
    For stablized the translation
    """
    string=string+'。'
    return string
# Tokenize a sentence
def clean_str(string, tolower=True):
    """
    Tokenization/string cleaning.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    #string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r'\(.*?\)', " \'d", string)
     # remove things in parathesesS
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    if tolower:
        string = string.lower()
    return string.strip()
def Process_clean(df,lg='zh'):
    if lg=='zh': 
        df.loc[:,"headline"] = df.headline.apply(lambda x: full_to_half(x))
        df.loc[:,"headline"] = df.headline.apply(lambda x: add_chinese_period (x))

    df.loc[:,"headline"] = df.headline.apply(lambda x: remove_special_chars(x))
    if lg=='zh':
        df.loc[:,"headline"] = df.headline.apply(lambda x: remove_space(x))
    return df

In [ ]:
# Tokenize a sentence
def clean_str(string, tolower=True):
    """
    Tokenization/string cleaning.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    #string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r'\(.*?\)', " \'d", string)
    # remove things in parathesesS
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    if tolower:
        string = string.lower()
    return string.strip()
#Calculate the length of the sentences so the batches will be patched efficiently
def compute_length(batch, text='headline'):
    return {
        'length': [len(item) for item in batch[text]]
    }
def sentence_embeddings(batch, model,text='translation'):
    with torch.no_grad():
        embeddings = model.encode(batch[text])
    return {'embeddings': embeddings}

In [ ]:
def cosinus_similarity (emb1, emb2):
# cosine similarity = normalize the vectors & multiply
    C = F.normalize(emb1) @ F.normalize(emb2).t()
    return C

In [ ]:
def cosine_similarity(a, b=None): 
    """ 
    cosine_similarity: Compute cosine similarity between two arrays. 
    If second array is None, computes similarity with self. 
    Args: a (np.array): dimension (n_a, p) b (np.array): dimension (n_b, p)
    """ 
    a /= np.sqrt(np.diag(a @ a.T))[:, None]
    if b is None: 
        b = a 
    else: 
        b /= np.sqrt(np.diag(b @ b.T))[:, None] 
    return a @ b.T

In [4]:
import json
# read the config file into a dictionary
with open("/home/yzhong/notebooks/getting_started/config_transformer.json", "r") as f:
    config_ = json.load(f)

print(config_)

{'TRANSLATION_MODEL': 'Helsinki-NLP/opus-mt-zh-en', 'SENTENCE_TRANSFORMER': 'all-MiniLM-L6-v2', 'model_dir': '/opt/tmp/yzhong/models/'}


## Prerequests: Read English and Chinese news from database

This notebooks requires a folder structure as this: 
- English_news_data/
  - 2011/
    - 01/
      - bloomberg_news_english_2011_01_01.parquet
      - bloomberg_news_english_2011_01_02.parquet
      - ...
    - 02/
      - bloomberg_news_english_2011_02_01.parquet
      - bloomberg_news_english_2011_02_02.parquet
      - ...
    - ...
      - ...
  - 2022/
    - 01/
      - bloomberg_news_english_2022_01_01.parquet
      - bloomberg_news_english_2022_01_02.parquet
      - ...
    - 02/
      - bloomberg_news_english_2022_02_01.parquet
      - bloomberg_news_english_2022_02_02.parquet
      - ...

Code for writing all the bloomberg English news into folder: 

In [ ]:
# Create the top-level directory for the English news data
if not os.path.exists("/mnt/live/user/yzhong/English_news_data"):
    os.mkdir("/mnt/live/user/yzhong/English_news_data")

for year in range(2011, 2023):
    year_str = str(year)

    # Create the year directory
    year_dir = os.path.join("/mnt/live/user/yzhong/English_news_data", year_str)
    if not os.path.exists(year_dir):
        os.mkdir(year_dir)

    for month in range(1, 13):
        month_str = f"{month:02d}"

        # Create the month directory
        month_dir = os.path.join(year_dir, month_str)
        if not os.path.exists(month_dir):
            os.mkdir(month_dir)

        # Determine the number of days in the month
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month == 2:
            if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0):
                num_days = 29
            else:
                num_days = 28
        else:
            num_days = 30

        for day in range(1, num_days+1):
            day_str = f"{day:02d}"

            # Construct the date string in the required format
            date_str = f"{year_str}/{month_str}/{day_str}"

            # Load the data for the current date
            df_b = get_bloomberg_english_news(date=date_str)

            # Save the data to a parquet file in the appropriate directory
            filename = f'bloomberg_news_english_{year_str}_{month_str}_{day_str}.parquet'
            filepath = os.path.join(month_dir, filename)
            df_b.to_parquet(filepath)

For the Chinese news, it was possible to write them all at once, so an extra seperation job was done. 

In [ ]:
# get the Chinese dataframe with trickers. 
#set the language to Chinese_simplified 
df_b = get_bloomberg_chinese_news(date="*/*/*")
df_b.to_parquet('/mnt/research-live/user/yzhong/bloomberg_news_chinese_trickers.parquet')

## Chinese news data

### Translation

Model: 

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu" #cuda:0 indique que tu veux être sur le 1er GPU, cuda:1 le 2e
MODEL_ID = config_['TRANSLATION_MODEL']
tokenizer = AutoTokenizer.from_pretrained( MODEL_ID, use_fast=True, cache_dir=config_['model_dir'] )
model = AutoModelForSeq2SeqLM.from_pretrained( MODEL_ID, cache_dir=config_['model_dir'] ).to(device)

Prepare the dataset

In [ ]:
df = pd.read_parquet("/mnt/research-live/user/yzhong/bloomberg_news_chinese.parquet")
df_c=df[df.language=='CHINESE_SIMP'] #prepare the dataframe for Chinese
df_c['headline_org']=df_c.loc[:,"headline"]
df_c.dropna(inplace=True)
df_c.reset_index(drop=True, inplace=True)
df_c=df_c.reset_index().rename(columns={'index': 'id'})
df_c=Process_clean(df_c, lg='zh')

Run the model 

In [ ]:
batch_size : int = 32
#Prepare dataset
ds = Dataset.from_dict(df_c)
ds = ds.map(lambda x: compute_length(x, text='headline'), batched=True).sort('length', reverse=True)
ds = ds.map(lambda x: translate(x, tokenizer, model, text='headline'), batched=True, batch_size=batch_size).sort('id')

Stock the dataset with translation

In [ ]:
ds.set_format("pandas")
df_c_t = ds[:]
df_c_t.to_parquet('/mnt/live/user/yzhong/bloomberg_news_chinese_translation.parquet')

### Embeddings

Prepare the dataset

In [ ]:
df_c=pd.read_parquet('/mnt/research-live/user/yzhong/bloomberg_news_chinese_trickers_translation.parquet')
df_c.dropna(subset=['headline'])
df_c.reset_index(drop=True, inplace=True)
df_c=df_c.reset_index().rename(columns={'index': 'id'})
df_c.loc[:,"translation"] = df_c.translation.apply(lambda x: clean_str(x))
ds_t = Dataset.from_dict(df_c)
cols_to_remove = ds_t.column_names
cols_to_remove.remove("translation")
cols_to_remove.remove("id")
ds_t1=ds_t.remove_columns(cols_to_remove)

Prepare the model 

In [ ]:
MODEL_ID = config_['SENTENCE_TRANSFORMER']
model = SentenceTransformer(MODEL_ID,device='cuda',cache_folder=config_['model_dir'] )
# Use the map method to apply the mapping function to the dataset in batches
batch_size =256*4

Obtain the embeddings

In [ ]:
ds_t1 = ds_t1.map(lambda x: compute_length(x, text='translation'), batched=True).sort('length', reverse=True)
ds_t1 = ds_t1.map(lambda x: sentence_embeddings(x ,model, text='translation'), batched=True, batch_size=batch_size).sort('id')

In [ ]:
translated_input_ids=ds_t1['embeddings']
df_c.to_parquet('/mnt/research-live/user/yzhong/bloomberg_news_chinese_trickers_translation_emb.parquet')

Write data into structured folders. 

This part of code allows to partition dataframe into daily frequency and have a same structure as English file.

In [ ]:
# Create the folder to store the monthly data
os.makedirs("/mnt/live/user/yzhong/Chinese_news_data_t", exist_ok=True)

# Read the data
df = pd.read_parquet("/mnt/research-live/user/yzhong/bloomberg_news_chinese_trickers_translation_emb.parquet")

# Extract year and month from the 'date' column
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day


for year in df['year'].unique():
    year_folder_path = os.path.join("/mnt/live/user/yzhong/Chinese_news_data_t", str(year))
    os.makedirs(year_folder_path, exist_ok=True)
    for month in df.loc[df['year'] == year, 'month'].unique():
        month_folder_path = os.path.join("/mnt/live/user/yzhong/Chinese_news_data_t",str(year), str(month))
        os.makedirs(month_folder_path, exist_ok=True)
        #print(month_folder_path)
        for day in df.loc[(df['year'] == year)&(df['month'] == month), 'day'].unique():
            #print(day)
            day_df = df.loc[(df['year'] == year) & (df['month'] == month)&(df['day'] == day)]
            del day_df['year']
            del day_df['month']
            del day_df['day']
            filename = f"bloomberg_news_chinese_{str(year)}_{str(month)}_{str(day)}.parquet"
            filepath = os.path.join(month_folder_path, filename)
            day_df.to_parquet(filepath)

## English news data

### Obtain the embeddings for all the English data

In [ ]:

folder_path = "/mnt/live/user/yzhong/English_news_data"
all_files = []
for year_folder in os.listdir(folder_path):
    year_path = os.path.join(folder_path, year_folder)
    if os.path.isdir(year_path):
        for month_folder in os.listdir(year_path):
            month_path = os.path.join(year_path, month_folder)
            if os.path.isdir(month_path):
                for file_name in os.listdir(month_path):
                    if file_name.endswith(".parquet"):
                        file_path = os.path.join(month_path, file_name)
                        df = pd.read_parquet(file_path)
                        df=df.reset_index().rename(columns={'index': 'id'})
                        df.loc[:,"headline"] = df.headline.apply(lambda x: clean_str(x))
                        ds = Dataset.from_dict(df)
                        cols_to_remove = ds.column_names
                        cols_to_remove.remove("headline")
                        cols_to_remove.remove("id")
                        ds_e=ds.remove_columns(cols_to_remove)
                        ds_e = ds_e.map(lambda x: compute_length(x, text='headline'), batched=True).sort('length', reverse=True)
                        ds_e = ds_e.map(lambda x: sentence_embeddings(x, model,text='headline'), batched=True, batch_size=batch_size).sort('id')
                        headline_input_ids=ds_e['embeddings']
                        df['embedding'] = headline_input_ids
                        df.to_parquet(file_path)


## Matching

In [12]:
df_c=pd.read_parquet('/mnt/research-live/user/yzhong/Chinese_news_data_t/2012/1/bloomberg_news_chinese_2012_1_10.parquet')
df_e=pd.read_parquet('/mnt/research-live/user/yzhong/English_news_data/2012/01/bloomberg_news_english_2012_01_10.parquet')

In [5]:
df_c=pd.read_parquet('/mnt/research-live/user/yzhong/Chinese_news_data_t/2018/11/bloomberg_news_chinese_2018_11_25.parquet')
df_e=pd.read_parquet('/mnt/research-live/user/yzhong/English_news_data/2018/11/bloomberg_news_english_2018_11_25.parquet')

NameError: name 'pd' is not defined

Define the logic for matching news

In [ ]:
from laserembeddings import Laser
laser = Laser()

In [146]:
matched_rows = []
df_c['match']=False
method='Laser'

for row in df_c.itertuples():
    ticker = row.tickers.split(', ')
    ticker=list(set(ticker))
    last_update=row.last_update
    con2 = (abs((last_update - df_e['last_update']).dt.total_seconds() / 60) < 120)
    #matches = df_e[df_e['tickers'].str.contains(ticker)]
    matches = df_e[df_e['tickers'].str.contains('|'.join(ticker)) & con2]
    matches = matches.reset_index(drop=True)
    if len(matches) > 0:
        df_c.at[row.Index, 'match'] = True
        matches['delay']=abs((last_update - df_e['last_update']).dt.total_seconds()) /60
        if len(matches) ==1:
            matching_headline = matches.headline.values[0]
            matching_delay = matches.delay.values[0]
            matching_date = matches.last_update.values[0]
            # df_c.at[row.Index, 'sim']=matches.headline.values[0]
            # df_c.at[row.Index, 'date']=matches.last_update.values[0]
        else: 
            if method=='Laser':
                chinese_sentences=row.headline
                english_sentences=matches.headline.to_list()
                chinese_emb=laser.embed_sentences(chinese_sentences,lang='zh')
                english_emb=laser.embed_sentences(english_sentences,lang='en')
                english_emb=torch.tensor(english_emb)
                chinese_emb=torch.tensor(chinese_emb)
                c=cosinus_similarity (chinese_emb, english_emb).numpy()
                
                
                sorted_indices = np.argsort(-c, axis=1)
            
                max_cos_indice = sorted_indices[0][0]
                
                matching_headline_cos = matches.at[max_cos_indice, 'headline']
                matching_date_cos = matches.at[max_cos_indice, 'last_update']
                matching_delay_cos = matches.at[max_cos_indice, 'delay']
                max_cos=c[0][max_cos_indice]
                df_c.at[row.Index, 'sim_cos'] = matching_headline_cos
                #df_c.at[row.Index, 'delay_cos'] = matching_delay_cos
                df_c.at[row.Index, 'cos'] = max_cos
                df_c.at[row.Index, 'date_cos'] = matching_date_cos
                
                #matches['delay']=abs((last_update - df_e['last_update']).dt.total_seconds())
                #matches['delay']
                min_delay_index = matches['delay'].idxmin()
                
                matching_headline = matches.at[min_delay_index, 'headline']
                #matching_delay = matches.at[min_delay_index, 'delay']
                matching_date = matches.at[min_delay_index, 'last_update']
        df_c.at[row.Index, 'sim'] = matching_headline
        #df_c.at[row.Index, 'delay'] = matching_delay
        df_c.at[row.Index, 'date'] = matching_date
                #print(matches['delay'])
#                 c=c.numpy()
# # Get the indices that would sort each row in descending order
# sorted_indices = np.argsort(-c, axis=1) # this will sort in descending values
# # Get the indices of the top three values for each row
# top_three_indices = sorted_indices[:, :3]
                
        
        
        # else: 


In [148]:

df_c.to_parquet('/mnt/research-live/user/yzhong/matching_2012_1_10.parquet')

In [149]:
df_c=pd.read_parquet('/mnt/research-live/user/yzhong/matching_2012_1_10.parquet')

In [151]:
with open('/mnt/research-live/user/yzhong/match_similarity.txt', 'w') as f:
    for row in df_c.itertuples():
        f.write(str(row.headline) + '\n')
        f.write(str(row.translation) + '\n')
        f.write(f' Sim: {row.delay} {row.sim}   ' + '\n')
        f.write(f' Sim_cos: {row.delay_cos} {row.sim_cos} ' + '\n')

In [ ]:
matched_rows = df_e[df_e['tickers'].isin(df_c['tickers'].tolist())]

In [13]:
match_dataset(df_c,df_e,method='Laser')

NameError: name 'laser' is not defined

## Process with all dataset

In [10]:
def match_dataset(df_c,df_e,method='Laser'):
    df_c['match']=False
    #method='Laser'

    for row in df_c.itertuples():
        ticker = row.tickers
        last_update=row.last_update
        con2 = (abs((last_update - df_e['last_update']).dt.total_seconds() / 60) < 120)
        #matches = df_e[df_e['tickers'].str.contains(ticker)]
        matches = df_e[df_e['tickers'].str.contains(ticker) & con2]
        matches = matches.reset_index(drop=True)
        if len(matches) > 0:
            df_c.at[row.Index, 'match'] = True
            matches['delay']=abs((last_update - df_e['last_update']).dt.total_seconds()) /60
            if len(matches) ==1:
                matching_headline = matches.headline.values[0]
                matching_delay = matches.delay.values[0]
                matching_date = matches.last_update.values[0]
                # df_c.at[row.Index, 'sim']=matches.headline.values[0]
                # df_c.at[row.Index, 'date']=matches.last_update.values[0]
            else: 
                if method=='Laser':
                    chinese_sentences=row.headline
                    english_sentences=matches.headline.to_list()
                    chinese_emb=laser.embed_sentences(chinese_sentences,lang='zh')
                    english_emb=laser.embed_sentences(english_sentences,lang='en')
                    english_emb=torch.tensor(english_emb)
                    chinese_emb=torch.tensor(chinese_emb)
                    c=cosinus_similarity (chinese_emb, english_emb).numpy()
                    
                    
                    sorted_indices = np.argsort(-c, axis=1)
                
                    max_cos_indice = sorted_indices[0][0]
                    
                    matching_headline_cos = matches.at[max_cos_indice, 'headline']
                    matching_date_cos = matches.at[max_cos_indice, 'last_update']
                    matching_delay_cos = matches.at[max_cos_indice, 'delay']
                    max_cos=c[0][max_cos_indice]
                    df_c.at[row.Index, 'sim_cos'] = matching_headline_cos
                    df_c.at[row.Index, 'delay_cos'] = matching_delay_cos
                    df_c.at[row.Index, 'cos'] = max_cos
                    df_c.at[row.Index, 'date_cos'] = matching_date_cos
                    
                    #matches['delay']=abs((last_update - df_e['last_update']).dt.total_seconds())
                    #matches['delay']
                    min_delay_index = matches['delay'].idxmin()
                    
                    matching_headline = matches.at[min_delay_index, 'headline']
                    matching_delay = matches.at[min_delay_index, 'delay']
                    matching_date = matches.at[min_delay_index, 'last_update']
            df_c.at[row.Index, 'sim'] = matching_headline
            df_c.at[row.Index, 'delay'] = matching_delay
            df_c.at[row.Index, 'date'] = matching_date

    return df_c

In [11]:
# Define the paths to the Chinese and English news data folders
chinese_folder_path = '/mnt/research-live/user/yzhong/Chinese_news_data_t'
english_folder_path = '/mnt/research-live/user/yzhong/English_news_data'

# Loop over the Chinese news data folders
for year_folder in os.listdir(chinese_folder_path):
    if os.path.isdir(os.path.join(chinese_folder_path, year_folder)):
        for month_folder in os.listdir(os.path.join(chinese_folder_path, year_folder)):
            for file_name in os.listdir(os.path.join(chinese_folder_path, year_folder,month_folder)):
            # Check if the file is a parquet file and contains the string 'bloomberg_news_chinese'
                if file_name.endswith('.parquet') and 'bloomberg_news_chinese' in file_name:
                   
                    df_c=pd.read_parquet(os.path.join(chinese_folder_path, year_folder,month_folder,file_name))
                    del df_c['id']
                    df_c=df_c.reset_index().rename(columns={'index': 'id'})
                    #get the file path for the english news
                    year = year_folder
                    month = month_folder
                    month_str = f"{int(month):02d}"
                    day=file_name.split('_')[-1].split('.')[0]
                    day_str=f"{int(day):02d}"
     
                    english_folder_year_month = os.path.join(english_folder_path, year, month_str)
                    if os.path.exists(english_folder_year_month):
                        english_file=os.path.join(english_folder_year_month, f'bloomberg_news_english_{year}_{month_str}_{day_str}.parquet')
                        df_e = pd.read_parquet(english_file) 
                        del df_e['id']
                        df_e=df_e.reset_index().rename(columns={'index': 'id'})
                    match_df=match_dataset(df_c,df_e,method='Laser')
                    # create the file path and name
                    file_name = f"/mnt/research-live/user/yzhong/Match_news/bloomberg_news_chinese_matching_{year}_{month_str}_{day_str}.parquet"
                    match_df.to_parquet(file_name)



ArrowInvalid: ('numpy.datetime64 scalars cannot be mixed with other Python scalar values currently', 'Conversion failed for column date with type object')

## Test with LASER

https://github.com/facebookresearch/LASER

In [ ]:
del df_c['id']
df_c=df_c.reset_index().rename(columns={'index': 'id'})
del df_e['id']
df_e=df_e.reset_index().rename(columns={'index': 'id'})

#### Using Laser

In [ ]:
from laserembeddings import Laser

laser = Laser()
chinese_sentences=df_c['headline'].to_list()
english_sentences=df_e['headline'].to_list()

In [ ]:
chinese_emb=laser.embed_sentences(chinese_sentences,lang='zh')
english_emb=laser.embed_sentences(english_sentences,lang='en')
english_emb=torch.tensor(english_emb)
chinese_emb=torch.tensor(chinese_emb)

#### Using translation embedding

In [ ]:
chinese_emb=torch.tensor(df_c['embedding'].to_list())
english_emb=torch.tensor(df_e['embedding'].to_list())

#### Calculate cosinus similarity

In [ ]:
c=cosinus_similarity (chinese_emb, english_emb)
c=c.numpy()
# Get the indices that would sort each row in descending order
sorted_indices = np.argsort(-c, axis=1) # this will sort in descending values
# Get the indices of the top three values for each row
top_three_indices = sorted_indices[:, :3]

In [ ]:
c.shape

(302, 8300)

In [ ]:
# Create an empty dataframe to store the three closest neighbours for each sentences
sim_df = pd.DataFrame(columns=['id', 'Sim1', 'Sim2', 'Sim3','tickers1','tickers2','tickers3', 'date1', 'date2', 'date3','cos1', 'cos2', 'cos3'])

# Loop over the rows of the `top_three_indices` array
for i, top_three in enumerate(top_three_indices):
    # Get the index of the current row
    index = i
    # Get the corresponding rows from `df_e`
    matches = df_e.loc[df_e['id'].isin(top_three)]
    matches = matches.reindex(index=top_three).reset_index(drop=True)
    
    # Create a new row to add to `result_df`
    new_row= {'id': index}
    
    # Loop over the matches and add their headlines and dates to the new row
    for j, match_row in matches.iterrows():
        new_row[f'Sim{j+1}'] = match_row['headline']
        new_row[f'date{j+1}'] = match_row['last_update']
        new_row[f'cos{j+1}'] = c[i][top_three[j]]
        new_row[f'tickers{j+1}'] = match_row['tickers']
    
    # Add the new row to `sim_df`
    sim_df = sim_df.append(new_row, ignore_index=True)

In [ ]:
merged_df = pd.merge(df_c, sim_df, on='id')
merged_df.to_parquet('/mnt/research-live/user/yzhong/Translation_similarity_2012_1_10.parquet')
df_sim=pd.read_parquet("/mnt/research-live/user/yzhong/Translation_similarity_2012_1_10.parquet")

In [ ]:
with open('/mnt/research-live/user/yzhong/Translation_similarity_2012_1_10.txt', 'w') as f:
    for row in merged_df.itertuples():
        f.write(str(row.headline) + '\n')
        f.write(str(row.tickers) + '\n')
        f.write(str(row.translation) + '\n')
        f.write(str(row.tickers1) + '\n')
        f.write(f' Sim1: {row.cos1}  {row.Sim1}' + '\n')
        f.write(str(row.tickers2) + '\n')
        f.write(f' Sim2: {row.cos2} {row.Sim2}' + '\n')
        f.write(str(row.tickers3) + '\n')
        f.write(f' Sim3: {row.cos3} {row.Sim3}' + '\n')

In [ ]:
def sim_dataframe (similarity_matrix,df_e,df_c):
    # Create an empty dataframe to store the three closest neighbours for each sentences
    sim_df = pd.DataFrame(columns=['id', 'Sim1', 'Sim2', 'Sim3', 'date1', 'date2', 'date3','cos1', 'cos2', 'cos3'])
    for i, dic in enumerate(similarity_matrix):
        index=i
        top_three=[dic[x]['corpus_id'] for x in range(3)]
        top_three_score=[dic[x]['score'] for x in range(3)]
        # Get the corresponding rows from `df_e`
        print(df_e.columns)
        matches = df_e.loc[df_e['id'].isin(top_three)]
        matches = matches.reindex(index=top_three).reset_index(drop=True)
        # Create a new row to add to `result_df`
        new_row= {'id': index}

        # Loop over the matches and add their headlines and dates to the new row
        for j, match_row in matches.iterrows():
            new_row[f'Sim{j+1}'] = match_row['headline']
            new_row[f'date{j+1}'] = match_row['last_update']
            new_row[f'cos{j+1}'] = top_three_score[j]
        
        # Add the new row to `sim_df`
        sim_df = sim_df.append(new_row, ignore_index=True)
        print(df_c.columns)
        merged_df = pd.merge(df_c, sim_df, on='id')

    return merged_df

In [ ]:
merged_df = pd.merge(df_c_t, sim_df, on='id')
merged_df.to_parquet('translated_similarity_2016_03_01.parquet')
df_sim=pd.read_parquet("translated_similarity_2016_03_01.parquet")

In [ ]:
with open('translated_similarity.txt', 'w') as f:
    for row in merged_df.itertuples():
        f.write(str(row.headline) + '\n')
        f.write(str(row.translation) + '\n')
        f.write(f' Sim1: {row.cos1}  {row.Sim1}' + '\n')
        f.write(f' Sim2: {row.cos2} {row.Sim2}' + '\n')
        f.write(f' Sim3: {row.cos3} {row.Sim3}' + '\n')

In [ ]:
chinese_emb.shape # shape1 sentences shape2 dimensions of embeddings 

(302, 1024)

Read data

In [ ]:
def sim_dataframe (similarity_matrix,df_e,df_c):
    # Create an empty dataframe to store the three closest neighbours for each sentences
    sim_df = pd.DataFrame(columns=['id', 'Sim1', 'Sim2', 'Sim3', 'date1', 'date2', 'date3','cos1', 'cos2', 'cos3'])
    for i, dic in enumerate(similarity_matrix):
        index=i
        top_three=[dic[x]['corpus_id'] for x in range(3)]
        top_three_score=[dic[x]['score'] for x in range(3)]
        # Get the corresponding rows from `df_e`
        print(df_e.columns)
        matches = df_e.loc[df_e['id'].isin(top_three)]
        matches = matches.reindex(index=top_three).reset_index(drop=True)
        # Create a new row to add to `result_df`
        new_row= {'id': index}

        # Loop over the matches and add their headlines and dates to the new row
        for j, match_row in matches.iterrows():
            new_row[f'Sim{j+1}'] = match_row['headline']
            new_row[f'date{j+1}'] = match_row['last_update']
            new_row[f'cos{j+1}'] = top_three_score[j]
        
        # Add the new row to `sim_df`
        sim_df = sim_df.append(new_row, ignore_index=True)
        print(df_c.columns)
        merged_df = pd.merge(df_c, sim_df, on='id')

    return merged_df

In [ ]:
def find_match(df1,df2):
    #df1 chinese df2 english
    #df1.dropna(inplace=True)
    #df1.reset_index(drop=True, inplace=True)
    #df1=df1.reset_index().rename(columns={'index': 'id'})
    #print(df1.columns)

    df2.dropna(inplace=True)
    df2.reset_index(drop=True, inplace=True)
    df2=df2.reset_index().rename(columns={'index': 'id'})

    df1.loc[:,"translation"] = df1.translation.apply(lambda x: clean_str(x))
    df2.loc[:,"headline"] = df2.headline.apply(lambda x: clean_str(x))
    ds_t = Dataset.from_dict(df1)
    ds_e = Dataset.from_dict(df2)
    cols_to_remove = ds_t.column_names
    cols_to_remove.remove("translation")
    cols_to_remove.remove("id")
    ds_t1=ds_t.remove_columns(cols_to_remove)
    cols_to_remove = ds_e.column_names
    cols_to_remove.remove("headline")
    cols_to_remove.remove("id")
    ds_e1=ds_e.remove_columns(cols_to_remove)
    ds_t1 = ds_t1.map(lambda x: compute_length(x, text='translation'), batched=True).sort('length', reverse=True)
    ds_t1 = ds_t1.map(lambda x: sentence_embeddings(x ,model, text='translation'), batched=True, batch_size=batch_size).sort('id')
    ds_e1 = ds_e1.map(lambda x: compute_length(x, text='headline'), batched=True).sort('length', reverse=True)
    ds_e1 = ds_e1.map(lambda x: sentence_embeddings(x, model,text='headline'), batched=True, batch_size=batch_size).sort('id')
    translated_input_ids=ds_t1['embeddings']
    originated_input_ids=ds_e1['embeddings']
    emb1=torch.tensor(translated_input_ids)
    emb2=torch.tensor(originated_input_ids)
    similarity_matrix=util.semantic_search(emb1, emb2, top_k=3)
    merge_df=sim_dataframe (similarity_matrix,df2,df1)
    return merge_df


In [ ]:
df_c=pd.read_parquet('/mnt/research-live/user/yzhong/Chinese news data/2011/bloomberg_news_chinese_2011_4.parquet')
import glob
import pandas as pd
file_list = glob.glob('/mnt/research-live/user/yzhong/English_news_data/2011/04/*.parquet')
dfs = []
for file_path in file_list:
    df = pd.read_parquet(file_path)
    dfs.append(df)
df_e = pd.concat(dfs, axis=0)

In [ ]:
df_c.loc[:,"translation"] = df_c.translation.apply(lambda x: clean_str(x))
df_e.loc[:,"headline"] = df_e.headline.apply(lambda x: clean_str(x))

In [ ]:
df_e.dropna(inplace=True)
df_e.reset_index(drop=True, inplace=True)
df_e=df_e.reset_index().rename(columns={'index': 'id'})

In [ ]:
ds_t = Dataset.from_dict(df_c)
ds_e = Dataset.from_dict(df_e)
ne=len(ds_e['headline'])
nt=len(ds_t['translation'])
print (f'English news: {ne}')
print (f'Chinese news: {nt}')

In [ ]:
cols_to_remove = ds_t.column_names
cols_to_remove.remove("translation")
cols_to_remove.remove("id")
ds_t1=ds_t.remove_columns(cols_to_remove)
ds_t1
cols_to_remove = ds_e.column_names
cols_to_remove.remove("headline")
cols_to_remove.remove("id")
ds_e1=ds_e.remove_columns(cols_to_remove)
ds_e1

In [ ]:
ds_t1 = ds_t1.map(lambda x: compute_length(x, text='translation'), batched=True).sort('length', reverse=True)
ds_t1 = ds_t1.map(lambda x: sentence_embeddings(x ,model, text='translation'), batched=True, batch_size=batch_size).sort('id')
ds_e1 = ds_e1.map(lambda x: compute_length(x, text='headline'), batched=True).sort('length', reverse=True)
ds_e1 = ds_e1.map(lambda x: sentence_embeddings(x, model,text='headline'), batched=True, batch_size=batch_size).sort('id')

In [ ]:
translated_input_ids=ds_t1['embeddings']
originated_input_ids=ds_e1['embeddings']
emb1=torch.tensor(translated_input_ids)
emb2=torch.tensor(originated_input_ids)
similarity_matrix=util.semantic_search(emb1, emb2, top_k=3)

In [ ]:
similarity_matrix

In [ ]:
# Create an empty dataframe to store the three closest neighbours for each sentences
sim_df = pd.DataFrame(columns=['id', 'Sim1', 'Sim2', 'Sim3', 'date1', 'date2', 'date3','cos1', 'cos2', 'cos3'])
for i, dic in enumerate(similarity_matrix):
    index=i
    top_three=[dic[x]['corpus_id'] for x in range(3)]
    top_three_score=[dic[x]['score'] for x in range(3)]
    # Get the corresponding rows from `df_e`
    matches = df_e.loc[df_e['id'].isin(top_three)]
    matches = matches.reindex(index=top_three).reset_index(drop=True)
    # Create a new row to add to `result_df`
    new_row= {'id': index}

    # Loop over the matches and add their headlines and dates to the new row
    for j, match_row in matches.iterrows():
        new_row[f'Sim{j+1}'] = match_row['headline']
        new_row[f'date{j+1}'] = match_row['last_update']
        new_row[f'cos{j+1}'] = top_three_score[j]
    
    # Add the new row to `sim_df`
    sim_df = sim_df.append(new_row, ignore_index=True)
    

In [ ]:
sim_df 

In [ ]:
import os
import pandas as pd

# Create the folder to store the monthly data
os.makedirs("/mnt/live/user/yzhong/Chinese_news_data_t", exist_ok=True)

# Read the data
df = pd.read_parquet("/mnt/research-live/user/yzhong/bloomberg_news_chinese_trickers_translation_emb.parquet")

# Extract year and month from the 'date' column
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day


for year in df['year'].unique():
    year_folder_path = os.path.join("/mnt/live/user/yzhong/Chinese_news_data_t", str(year))
    os.makedirs(year_folder_path, exist_ok=True)
    for month in df.loc[df['year'] == year, 'month'].unique():
        month_folder_path = os.path.join("/mnt/live/user/yzhong/Chinese_news_data_t",str(year), str(month))
        os.makedirs(month_folder_path, exist_ok=True)
        #print(month_folder_path)
        for day in df.loc[(df['year'] == year)&(df['month'] == month), 'day'].unique():
            #print(day)
            day_df = df.loc[(df['year'] == year) & (df['month'] == month)&(df['day'] == day)]
            del day_df['year']
            del day_df['month']
            del day_df['day']
            filename = f"bloomberg_news_chinese_{str(year)}_{str(month)}_{str(day)}.parquet"
            filepath = os.path.join(month_folder_path, filename)
            day_df.to_parquet(filepath)
